In [9]:
!export AUTOGRAPH_VERBOSITY=0
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False
import tensorflow.python.util.deprecation as deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False
import tensorflow as tf 
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
# tf.compat.v1.logging.set_verbosity(100)

In [10]:
%load_ext autoreload
%autoreload 2
import cantera as ct
import os
import gym 
import numpy as np 
import pandas as pd
from stable_baselines.common.policies import MlpPolicy, CnnLstmPolicy,MlpLstmPolicy
from stable_baselines.common.vec_env import SubprocVecEnv
from stable_baselines.bench import Monitor
from stable_baselines.results_plotter import load_results, ts2xy
from stable_baselines.common import set_global_seeds
from stable_baselines import ACKTR, PPO2
from envs.sim_env import SimEnv, NO_idx, CO_idx, O2_idx, H2O_idx
from envs.SimUtils import solvePhi_airSplit, equil, runMainBurner, correctNOx
import seaborn as sns

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
model = PPO2.load('Trained Models/PPO2_MlpPolicy_11172019_1000000.zip')

In [6]:
from stable_baselines.common.vec_env import DummyVecEnv
sim_env = SimEnv()
env = DummyVecEnv([lambda: sim_env])
# env = SimEnv()
obs = env.reset()
env.action_space.high

array([1., 1., 1.])

In [11]:
import pdb 
game_over = False
results_list = []
while not game_over: 
#     action, _states = model.predict(obs)
    action = env.action_space.high[np.newaxis,:]
#     action = env.action_space.high[np.newaxis,:]
    obs, rewards, game_over, info = env.step(action)
    env.render(mode='human')
    g = env.envs[0].sec_stage_gas
    results_list.append([
        env.envs[0].age/1e-3, 
        g.T, 
        correctNOx(g.X[NO_idx], g.X[H2O_idx], g.X[O2_idx]),
        correctNOx(g.X[CO_idx], g.X[H2O_idx], g.X[O2_idx]),
        env.envs[0].mfc_main.mdot(0),
        env.envs[0].mfc_fuel_sec.mdot(0),
        env.envs[0].mfc_air_sec.mdot(0), 
        env.envs[0].remaining_main_burner_mass,
        env.envs[0].sec_fuel_remaining,
        env.envs[0].sec_air_remaining,
        rewards, ])

2| 0.00| 1081.86| 0.58| 0.05| 0.89| 88.65| 1.69| 9.63| 17733.38| 1690.62| 9642.50| 1.00| 1.00| 1.00| 211.31|0.00| 11.82|199.50|0.00
3| 0.00| 1081.93| 0.58| 0.05| 0.89| 88.63| 1.69| 9.62| 17729.83| 1688.93| 9632.85| 1.00| 1.00| 1.00| 211.32|0.00| 11.82|199.50|0.00
4| 0.00| 1082.01| 0.58| 0.05| 0.89| 88.61| 1.69| 9.61| 17726.28| 1687.24| 9623.22| 1.00| 1.00| 1.00| 211.32|0.01| 11.82|199.50|0.00
5| 0.00| 1082.09| 0.58| 0.05| 0.89| 88.60| 1.68| 9.60| 17722.74| 1685.56| 9613.60| 1.00| 1.00| 1.00| 211.32|0.01| 11.82|199.50|0.00
6| 0.01| 1082.18| 0.58| 0.05| 0.89| 88.58| 1.68| 9.59| 17719.19| 1683.87| 9603.98| 1.00| 1.00| 1.00| 211.32|0.02| 11.83|199.50|0.00
7| 0.01| 1082.27| 0.58| 0.05| 0.89| 88.56| 1.68| 9.58| 17715.65| 1682.19| 9594.38| 1.00| 1.00| 1.00| 211.32|0.03| 11.83|199.50|0.00
8| 0.01| 1082.36| 0.58| 0.05| 0.89| 88.54| 1.68| 9.58| 17712.11| 1680.51| 9584.79| 1.00| 1.00| 1.00| 211.33|0.04| 11.83|199.50|0.00
9| 0.01| 1082.45| 0.58| 0.05| 0.89| 88.53| 1.68| 9.57| 17708.56| 1678.82| 95

86| 0.09| 1089.42| 0.58| 0.05| 0.86| 87.17| 1.55| 8.86| 17437.91| 1554.35| 8865.24| 1.00| 1.00| 1.00| 211.50|5.85| 12.00|199.50|0.00
87| 0.09| 1089.51| 0.58| 0.05| 0.86| 87.15| 1.55| 8.85| 17434.43| 1552.79| 8856.38| 1.00| 1.00| 1.00| 211.50|5.99| 12.00|199.50|0.00
88| 0.09| 1089.60| 0.58| 0.05| 0.86| 87.14| 1.55| 8.84| 17430.94| 1551.24| 8847.52| 1.00| 1.00| 1.00| 211.50|6.12| 12.00|199.50|0.00
89| 0.09| 1089.69| 0.58| 0.05| 0.86| 87.12| 1.55| 8.83| 17427.45| 1549.69| 8838.67| 1.00| 1.00| 1.00| 211.50|6.26| 12.01|199.50|0.00
90| 0.09| 1089.78| 0.58| 0.05| 0.85| 87.10| 1.55| 8.82| 17423.97| 1548.14| 8829.83| 1.00| 1.00| 1.00| 211.50|6.40| 12.01|199.50|0.00
91| 0.09| 1089.87| 0.58| 0.05| 0.85| 87.08| 1.55| 8.81| 17420.48| 1546.59| 8821.00| 1.00| 1.00| 1.00| 211.51|6.54| 12.01|199.50|0.00
92| 0.09| 1089.96| 0.58| 0.05| 0.85| 87.07| 1.54| 8.80| 17417.00| 1545.04| 8812.18| 1.00| 1.00| 1.00| 211.51|6.69| 12.01|199.50|0.00
93| 0.09| 1090.05| 0.58| 0.05| 0.85| 87.05| 1.54| 8.79| 17413.52| 154

147| 0.15| 1094.84| 0.58| 0.05| 0.84| 86.11| 1.46| 8.33| 17226.44| 1462.32| 8340.37| 1.00| 1.00| 1.00| 211.62|16.75| 12.13|199.50|0.00
148| 0.15| 1094.93| 0.58| 0.05| 0.84| 86.10| 1.46| 8.32| 17223.00| 1460.86| 8332.03| 1.00| 1.00| 1.00| 211.63|16.97| 12.13|199.50|0.00
149| 0.15| 1095.02| 0.58| 0.05| 0.84| 86.08| 1.46| 8.32| 17219.55| 1459.40| 8323.70| 1.00| 1.00| 1.00| 211.63|17.19| 12.14|199.50|0.00
150| 0.15| 1095.10| 0.58| 0.05| 0.84| 86.06| 1.46| 8.31| 17216.11| 1457.94| 8315.37| 1.00| 1.00| 1.00| 211.63|17.42| 12.14|199.50|0.00
151| 0.15| 1095.19| 0.58| 0.05| 0.84| 86.05| 1.46| 8.30| 17212.67| 1456.48| 8307.06| 1.00| 1.00| 1.00| 211.63|17.64| 12.14|199.50|0.00
152| 0.15| 1095.28| 0.58| 0.05| 0.84| 86.03| 1.45| 8.29| 17209.22| 1455.02| 8298.75| 1.00| 1.00| 1.00| 211.63|17.87| 12.14|199.50|0.00
153| 0.15| 1095.37| 0.58| 0.05| 0.84| 86.01| 1.45| 8.28| 17205.78| 1453.57| 8290.45| 1.00| 1.00| 1.00| 211.64|18.10| 12.14|199.50|0.00
154| 0.15| 1095.45| 0.58| 0.05| 0.84| 85.99| 1.45| 8.27

237| 0.24| 1102.67| 0.57| 0.05| 0.84| 84.58| 1.34| 7.61| 16919.11| 1336.40| 7622.18| 1.00| 1.00| 1.00| 211.81|41.88| 12.32|199.50|0.00
238| 0.24| 1102.76| 0.57| 0.05| 0.84| 84.56| 1.33| 7.61| 16915.73| 1335.06| 7614.56| 1.00| 1.00| 1.00| 211.81|42.22| 12.33|199.50|0.00
239| 0.24| 1102.84| 0.57| 0.05| 0.84| 84.54| 1.33| 7.60| 16912.34| 1333.73| 7606.94| 1.00| 1.00| 1.00| 211.81|42.56| 12.33|199.50|0.00
240| 0.24| 1102.93| 0.57| 0.05| 0.84| 84.53| 1.33| 7.59| 16908.96| 1332.40| 7599.34| 1.00| 1.00| 1.00| 211.81|42.89| 12.33|199.50|0.00
241| 0.24| 1103.02| 0.57| 0.05| 0.84| 84.51| 1.33| 7.58| 16905.58| 1331.06| 7591.74| 1.00| 1.00| 1.00| 211.81|43.23| 12.33|199.50|0.00
242| 0.24| 1103.10| 0.57| 0.05| 0.84| 84.49| 1.33| 7.58| 16902.20| 1329.73| 7584.15| 1.00| 1.00| 1.00| 211.82|43.57| 12.33|199.50|0.00
243| 0.24| 1103.19| 0.57| 0.05| 0.84| 84.48| 1.33| 7.57| 16898.82| 1328.40| 7576.56| 1.00| 1.00| 1.00| 211.82|43.91| 12.34|199.50|0.00
244| 0.24| 1103.27| 0.57| 0.05| 0.84| 84.46| 1.33| 7.56

326| 0.33| 1110.23| 0.57| 0.05| 0.86| 83.09| 1.22| 6.97| 16620.59| 1222.55| 6972.81| 1.00| 1.00| 1.00| 211.97|76.15| 12.51|199.50|0.00
327| 0.33| 1110.32| 0.57| 0.05| 0.86| 83.07| 1.22| 6.96| 16617.26| 1221.32| 6965.83| 1.00| 1.00| 1.00| 211.97|76.59| 12.51|199.50|0.00
328| 0.33| 1110.40| 0.57| 0.05| 0.86| 83.05| 1.22| 6.95| 16613.94| 1220.10| 6958.87| 1.00| 1.00| 1.00| 211.98|77.02| 12.51|199.50|0.00
329| 0.33| 1110.49| 0.57| 0.05| 0.86| 83.04| 1.22| 6.94| 16610.62| 1218.88| 6951.91| 1.00| 1.00| 1.00| 211.98|77.46| 12.52|199.50|0.00
330| 0.33| 1110.57| 0.57| 0.05| 0.86| 83.02| 1.22| 6.94| 16607.29| 1217.66| 6944.96| 1.00| 1.00| 1.00| 211.98|77.89| 12.52|199.50|0.00
331| 0.33| 1110.65| 0.57| 0.05| 0.86| 83.00| 1.22| 6.93| 16603.97| 1216.45| 6938.01| 1.00| 1.00| 1.00| 211.98|78.33| 12.52|199.50|0.00
332| 0.33| 1110.74| 0.57| 0.05| 0.86| 82.99| 1.21| 6.92| 16600.65| 1215.23| 6931.08| 1.00| 1.00| 1.00| 211.98|78.77| 12.52|199.50|0.00
333| 0.33| 1110.82| 0.57| 0.05| 0.86| 82.97| 1.21| 6.92

...

15684| 15.68| 1996.79| -0.00| 222.09| 32.29| 3.85| 0.00| 0.00| 770.08| 0.00| 0.00| 1.00| 1.00| 1.00| -3710.05|9244.90| 49.03|0.00|98.99
15685| 15.68| 1996.79| -0.00| 222.09| 32.28| 3.85| 0.00| 0.00| 769.92| 0.00| 0.00| 1.00| 1.00| 1.00| -3710.79|9245.04| 49.03|0.00|98.99
15686| 15.69| 1996.79| -0.00| 222.09| 32.28| 3.85| 0.00| 0.00| 769.77| 0.00| 0.00| 1.00| 1.00| 1.00| -3711.52|9245.19| 49.03|0.00|99.00
15687| 15.69| 1996.78| -0.00| 222.09| 32.28| 3.85| 0.00| 0.00| 769.61| 0.00| 0.00| 1.00| 1.00| 1.00| -3712.25|9245.34| 49.03|0.00|99.00
15688| 15.69| 1996.78| -0.00| 222.09| 32.28| 3.85| 0.00| 0.00| 769.46| 0.00| 0.00| 1.00| 1.00| 1.00| -3712.99|9245.49| 49.03|0.00|99.00
15689| 15.69| 1996.78| -0.00| 222.10| 32.28| 3.85| 0.00| 0.00| 769.31| 0.00| 0.00| 1.00| 1.00| 1.00| -3713.72|9245.64| 49.03|0.00|99.01
15690| 15.69| 1996.77| -0.00| 222.10| 32.28| 3.84| 0.00| 0.00| 769.15| 0.00| 0.00| 1.00| 1.00| 1.00| -3714.46|9245.78| 49.03|0.00|99.01
15691| 15.69| 1996.77| -0.00| 222.10| 32.28| 3.8

15754| 15.75| 1996.54| -0.00| 222.23| 32.21| 3.80| 0.00| 0.00| 759.37| 0.00| 0.00| 1.00| 1.00| 1.00| -3761.72|9255.19| 49.05|0.00|99.20
15755| 15.75| 1996.53| -0.00| 222.23| 32.21| 3.80| 0.00| 0.00| 759.22| 0.00| 0.00| 1.00| 1.00| 1.00| -3762.46|9255.34| 49.05|0.00|99.20
15756| 15.76| 1996.53| -0.00| 222.24| 32.21| 3.79| 0.00| 0.00| 759.07| 0.00| 0.00| 1.00| 1.00| 1.00| -3763.20|9255.48| 49.05|0.00|99.20
15757| 15.76| 1996.53| -0.00| 222.24| 32.21| 3.79| 0.00| 0.00| 758.91| 0.00| 0.00| 1.00| 1.00| 1.00| -3763.94|9255.63| 49.05|0.00|99.20
15758| 15.76| 1996.52| -0.00| 222.24| 32.20| 3.79| 0.00| 0.00| 758.76| 0.00| 0.00| 1.00| 1.00| 1.00| -3764.69|9255.78| 49.05|0.00|99.21
15759| 15.76| 1996.52| -0.00| 222.24| 32.20| 3.79| 0.00| 0.00| 758.61| 0.00| 0.00| 1.00| 1.00| 1.00| -3765.43|9255.92| 49.05|0.00|99.21
15760| 15.76| 1996.51| -0.00| 222.24| 32.20| 3.79| 0.00| 0.00| 758.46| 0.00| 0.00| 1.00| 1.00| 1.00| -3766.17|9256.07| 49.05|0.00|99.21
15761| 15.76| 1996.51| -0.00| 222.25| 32.20| 3.7

15827| 15.83| 1996.27| -0.00| 222.38| 32.13| 3.74| 0.00| 0.00| 748.36| 0.00| 0.00| 1.00| 1.00| 1.00| -3816.14|9265.78| 49.06|0.00|99.36
15828| 15.83| 1996.27| -0.00| 222.39| 32.13| 3.74| 0.00| 0.00| 748.21| 0.00| 0.00| 1.00| 1.00| 1.00| -3816.89|9265.93| 49.06|0.00|99.37
15829| 15.83| 1996.27| -0.00| 222.39| 32.13| 3.74| 0.00| 0.00| 748.06| 0.00| 0.00| 1.00| 1.00| 1.00| -3817.64|9266.07| 49.06|0.00|99.37
15830| 15.83| 1996.26| -0.00| 222.39| 32.13| 3.74| 0.00| 0.00| 747.91| 0.00| 0.00| 1.00| 1.00| 1.00| -3818.39|9266.21| 49.06|0.00|99.37
15831| 15.83| 1996.26| -0.00| 222.39| 32.13| 3.74| 0.00| 0.00| 747.76| 0.00| 0.00| 1.00| 1.00| 1.00| -3819.14|9266.36| 49.06|0.00|99.37
15832| 15.83| 1996.26| -0.00| 222.39| 32.12| 3.74| 0.00| 0.00| 747.61| 0.00| 0.00| 1.00| 1.00| 1.00| -3819.89|9266.50| 49.06|0.00|99.37
15833| 15.83| 1996.25| -0.00| 222.40| 32.12| 3.74| 0.00| 0.00| 747.46| 0.00| 0.00| 1.00| 1.00| 1.00| -3820.64|9266.65| 49.06|0.00|99.38
15834| 15.83| 1996.25| -0.00| 222.40| 32.12| 3.7

15899| 15.90| 1996.02| -0.00| 222.54| 32.05| 3.69| 0.00| 0.00| 737.66| 0.00| 0.00| 1.00| 1.00| 1.00| -3870.35|9276.08| 49.08|0.00|99.49
15900| 15.90| 1996.01| -0.00| 222.54| 32.05| 3.69| 0.00| 0.00| 737.51| 0.00| 0.00| 1.00| 1.00| 1.00| -3871.11|9276.23| 49.08|0.00|99.50
15901| 15.90| 1996.01| -0.00| 222.54| 32.05| 3.69| 0.00| 0.00| 737.37| 0.00| 0.00| 1.00| 1.00| 1.00| -3871.86|9276.37| 49.08|0.00|99.50
15902| 15.90| 1996.01| -0.00| 222.54| 32.05| 3.69| 0.00| 0.00| 737.22| 0.00| 0.00| 1.00| 1.00| 1.00| -3872.62|9276.51| 49.08|0.00|99.50
15903| 15.90| 1996.00| -0.00| 222.54| 32.05| 3.68| 0.00| 0.00| 737.07| 0.00| 0.00| 1.00| 1.00| 1.00| -3873.38|9276.65| 49.08|0.00|99.50
15904| 15.90| 1996.00| -0.00| 222.55| 32.05| 3.68| 0.00| 0.00| 736.92| 0.00| 0.00| 1.00| 1.00| 1.00| -3874.13|9276.79| 49.08|0.00|99.50
15905| 15.90| 1996.00| -0.00| 222.55| 32.05| 3.68| 0.00| 0.00| 736.78| 0.00| 0.00| 1.00| 1.00| 1.00| -3874.89|9276.94| 49.08|0.00|99.50
15906| 15.91| 1995.99| -0.00| 222.55| 32.05| 3.6

15976| 15.98| 1995.75| -0.00| 222.70| 31.97| 3.63| 0.00| 0.00| 726.39| 0.00| 0.00| 1.00| 1.00| 1.00| -3928.90|9286.94| 49.09|0.00|99.60
15977| 15.98| 1995.74| -0.00| 222.70| 31.97| 3.63| 0.00| 0.00| 726.24| 0.00| 0.00| 1.00| 1.00| 1.00| -3929.67|9287.08| 49.09|0.00|99.60
15978| 15.98| 1995.74| -0.00| 222.70| 31.97| 3.63| 0.00| 0.00| 726.10| 0.00| 0.00| 1.00| 1.00| 1.00| -3930.43|9287.22| 49.09|0.00|99.60
15979| 15.98| 1995.74| -0.00| 222.70| 31.97| 3.63| 0.00| 0.00| 725.95| 0.00| 0.00| 1.00| 1.00| 1.00| -3931.20|9287.36| 49.09|0.00|99.61
15980| 15.98| 1995.73| -0.00| 222.70| 31.97| 3.63| 0.00| 0.00| 725.81| 0.00| 0.00| 1.00| 1.00| 1.00| -3931.96|9287.50| 49.09|0.00|99.61
15981| 15.98| 1995.73| -0.00| 222.71| 31.97| 3.63| 0.00| 0.00| 725.66| 0.00| 0.00| 1.00| 1.00| 1.00| -3932.72|9287.64| 49.09|0.00|99.61
15982| 15.98| 1995.73| -0.00| 222.71| 31.97| 3.63| 0.00| 0.00| 725.52| 0.00| 0.00| 1.00| 1.00| 1.00| -3933.49|9287.78| 49.09|0.00|99.61
15983| 15.98| 1995.72| -0.00| 222.71| 31.97| 3.6

In [ ]:
df = pd.DataFrame(data=np.vstack(results_list), columns=['age', 'T', 'NO', 'CO', 'mdot_main', 'mdot_fuel_sec', 'mdot_air_sec', 'rem_main', 'rem_fuel_sec', 'rem_air_sec', 'reward'])
df = df.iloc[0:-1]
df.to_csv("simResults_500000.csv", index=False)

In [ ]:
df

In [ ]:
df['mdot_main'].hist()

In [ ]:
df.plot('age', 'mdot_fuel_sec')

In [ ]:
env.envs[0].action_space.high[2]